In [1]:
from wave_net_utils import load_voices_files, get_max_length,fill_voices_data_with_pads, quantize_voices, get_voices_labels
import torch
import numpy as np

voices = load_voices_files(['dataset/dev/LJ050-0274.wav', 'dataset/dev/LJ050-0275.wav'])

voices_with_pads, lengths = fill_voices_data_with_pads(voices)

print('voices lengths:', len(voices[0]), len(voices[1]))
print('voices lengths:', lengths)
print('voices_with_pads lengths:', len(voices_with_pads[0]), len(voices_with_pads[1]))
# print(voices_with_pads)
voices_with_pads = np.array(voices_with_pads) / (65536/2)
print(voices_with_pads)
labels = get_voices_labels(voices_with_pads, 256)
# quantized_voices = quantize_voices(voices_with_pads, 256)
# print(quantized_voices)
print(np.max(labels))
print(np.min(labels))


voices lengths: 99229 201117
voices lengths: [99229, 201117]
voices_with_pads lengths: 201117 201117
[[-1.83105469e-04 -2.44140625e-04 -2.44140625e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 9.15527344e-05  2.74658203e-04  1.83105469e-04 ...  3.66210938e-04
  -6.10351562e-05 -3.05175781e-05]]
244
20


In [1]:
import torch
from wave_net import WaveNet
model = WaveNet(30, layer_channels=16)
model.train()
# device = torch.device("cuda:0")
device = torch.device("cpu") 
print('use device: %s' % device, file=sys.stderr)
model = model.to(device)


use device: cpu


In [3]:
from wave_net_utils import load_voices_files, get_voices_labels
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm

voices = load_voices_files(['dataset/dev/LJ050-0274.wav', 'dataset/dev/LJ050-0275.wav'])


labels = get_voices_labels(voices, 256)
labels = torch.from_numpy(labels).long()
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10): 
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(voices, None)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print('[%d] loss: %.3f' % (epoch + 1, loss.item() ))



[1] loss: 5.511
[2] loss: 5.506
[3] loss: 5.487
[4] loss: 5.477
[5] loss: 5.458
[6] loss: 5.439
[7] loss: 5.432
[8] loss: 5.413
[9] loss: 5.409
[10] loss: 5.391
